In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [2]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.block1 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AvgPool2d(2)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AvgPool2d(2)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AvgPool2d(2)
        )
        #self.final_conv = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.final_conv = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.mean_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512, 10)

    def forward(self, x, extract_layer=None, mode="features"):
        x = self.conv1(x)
        if extract_layer == 1: return x
        x = self.block1(x)
        if extract_layer == 2: return x
        x = self.block2(x)
        if extract_layer == 3: return x
        x = self.block3(x)
        if extract_layer == 4: return x
        x = self.final_conv(x)
        if extract_layer == 5: return x
        x = self.mean_pool(x).view(x.size(0), -1)
        if extract_layer == 6: return x

        if mode == "classify":
            return self.fc(x)

        return x


In [3]:
cifar_mean = [0.4914, 0.4822, 0.4465]
cifar_std = [0.2470, 0.2435, 0.2616]

basic_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(cifar_mean, cifar_std)])

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(cifar_mean, cifar_std)
])

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=basic_transform)
d_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=basic_transform)

d_test = torch.utils.data.Subset(d_test, range(len(d_test)))

# d_train, d_aux = torch.utils.data.random_split(dataset, [25000, 25000])

# img, label = d_train[0]
# print(img.shape)
# print(label)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
indices = list(range(len(dataset)))
split = 25000

# First 25,000 samples for d_train, the rest for d_aux
d_train = torch.utils.data.Subset(dataset, indices[:split])
d_aux = torch.utils.data.Subset(dataset, indices[split:])

In [5]:
def train_for_classification(model, dataloader, epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model.to(device)

    params = [
    {'params': [p for name, p in model.named_parameters() if 'fc' not in name], 'weight_decay': 0.0},
    {'params': model.fc.parameters(), 'weight_decay': 0.01}
    ]

    optimiser = optim.Adam(params, lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimiser.zero_grad()

            outputs = model(images, mode="classify")
            loss = criterion(outputs, labels)
            loss.backward()
            optimiser.step()

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        print(f"Epoch {epoch}: Loss: {total_loss / len(dataloader)}, Accuracy: {100 * correct/total:.2f}%")
        

def train_cnn(model, dataloader, epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model.to(device)
    optimiser = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimiser.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimiser.step()

In [6]:
def predict(model, dataloader):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    predictions = []
    with torch.no_grad():
        for images, _ in dataloader:
            images = images.to(device)
            outputs = model(images, mode="classify")
            _, predicted = outputs.max(1)
            predictions.extend(predicted.cpu().numpy())
    return predictions

def evaluate_accuracy(model, dataloader):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images, mode="classify")
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy

In [7]:
def initialize_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)

In [8]:
dataloader_train = torch.utils.data.DataLoader(d_train, batch_size=64, shuffle=True)
dataloader_test = torch.utils.data.DataLoader(d_test, batch_size=64, shuffle=True)
train_data_cnn = ConvNet()
# train_data_cnn.apply(initialize_weights)
train_for_classification(train_data_cnn, dataloader_train, epochs=12)
evaluate_accuracy(train_data_cnn, dataloader_test)

cpu


Epoch 0: Loss: 1.4815018518501535, Accuracy: 45.31%
Epoch 1: Loss: 1.068536635859848, Accuracy: 61.53%
Epoch 2: Loss: 0.8923222687848084, Accuracy: 68.55%
Epoch 3: Loss: 0.7559754484144928, Accuracy: 73.28%
Epoch 4: Loss: 0.6464797538869521, Accuracy: 77.18%
Epoch 5: Loss: 0.5666638731651599, Accuracy: 80.57%
Epoch 6: Loss: 0.48961950811888555, Accuracy: 83.00%
Epoch 7: Loss: 0.42667255731647274, Accuracy: 85.22%
Epoch 8: Loss: 0.3661833546122017, Accuracy: 87.44%
Epoch 9: Loss: 0.31554088732013313, Accuracy: 89.22%
Epoch 10: Loss: 0.2607108767494521, Accuracy: 91.04%
Epoch 11: Loss: 0.23193407851411862, Accuracy: 92.16%


80.98

In [9]:
evaluate_accuracy(train_data_cnn, dataloader_test)

80.98

In [10]:
# was 81.37 without weight decay (l2 regularization)